# Cropping outsider

In [1]:
import numpy as np
import cv2

In [2]:
body_cascade = cv2.CascadeClassifier('haarcascade_fullbody.xml')

SystemError: <class 'cv2.CascadeClassifier'> returned a result with an error set

In [ ]:
input_img=cv2.imread('Vriksasana.jpeg', cv2.IMREAD_GRAYSCALE) # get the array of the original pic
print(input_img)
faces = body_cascade.detectMultiScale(input_img, 
                                      scaleFactor=1.1,
                                      minNeighbors=5)
input_img.shape



In [ ]:
x,y,w,h = faces[0]
img_data= input_img[y:y+h,x:x+w]
img_data=cv2.resize(img_data,(48,48))
    
img_data = np.stack(img_data)
img_data = img_data / 255.0

In [ ]:
import numpy as np
import tensorflow as tf
import cv2 as cv
from pathlib import Path
from send2trash import send2trash
from google_images_download import google_images_download

In [ ]:
# Read the graph.
with tf.gfile.GFile('./ssdlite_mobilenet_v2_coco_2018_05_09/frozen_inference_graph.pb', 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())

In [ ]:
def detect_persons_and_save_crops(asana, graph_def, folder):
    with tf.Session() as sess:
        # Restore session
        sess.graph.as_default()
        tf.import_graph_def(graph_def, name='ejemplo acostado')

        files = list(Path(folder).glob('*.jpg'))
        files.extend(list(Path(folder).glob('*.png')))

        for fid,fname in enumerate(files): 
            # Read and preprocess an image.
            img = cv.imread(str(fname))
            if(img is not None):
                rows = img.shape[0]
                cols = img.shape[1]
                inp = cv.resize(img, (300, 300))
                inp = inp[:, :, [2, 1, 0]]  # BGR2RGB

                # Run the model
                out = sess.run([sess.graph.get_tensor_by_name('num_detections:0'),
                                sess.graph.get_tensor_by_name('detection_scores:0'),
                                sess.graph.get_tensor_by_name('detection_boxes:0'),
                                sess.graph.get_tensor_by_name('detection_classes:0')],
                               feed_dict={'image_tensor:0': inp.reshape(1, inp.shape[0], inp.shape[1], 3)})

                # Visualize detected bounding boxes.
                num_detections = int(out[0][0])
                #print(fname, num_detections)
                for i in range(num_detections):
                    classId = int(out[3][0][i])
                    if(classId==1):
                        score = float(out[1][0][i])
                        bbox = [float(v) for v in out[2][0][i]]
                        if score > 0.3:
                            x = int(bbox[1] * cols)
                            y = int(bbox[0] * rows)
                            right = int(bbox[3] * cols)
                            bottom = int(bbox[2] * rows)
                            cv.imwrite(folder+str(fid)+'-'+str(i)+'.png', img[y:bottom, x:right])

            try:
                send2trash(str(fname))
            except: pass    
        print('cleaning '+asana+' done.')
    return

In [ ]:
import cv2
#reading the image 
image = cv2.imread("Vriksasana.jpg")
edged = cv2.Canny(image, 10, 250)
cv2.imshow("Edges", edged)
cv2.waitKey(0)
 
#applying closing function 
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7, 7))
closed = cv2.morphologyEx(edged, cv2.MORPH_CLOSE, kernel)
cv2.imshow("Closed", closed)
cv2.waitKey(0)
 
#finding_contours 
(cnts, _) = cv2.findContours(closed.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
 
for c in cnts:
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)
    cv2.drawContours(image, [approx], -1, (0, 255, 0), 2)
cv2.imshow("Output", image)
cv2.waitKey(0)
